# Data Preprocessing

In [2]:
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn import set_config
import joblib

In [3]:
path = '../../data/raw/coffee_shop_sales.csv'
df = pd.read_csv(path)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149116 entries, 0 to 149115
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   transaction_id    149116 non-null  int64  
 1   transaction_date  149116 non-null  object 
 2   transaction_time  149116 non-null  object 
 3   store_location    149116 non-null  object 
 4   product_category  149116 non-null  object 
 5   product_type      149116 non-null  object 
 6   product_detail    149116 non-null  object 
 7   product_size      149116 non-null  object 
 8   unit_price        149116 non-null  float64
 9   quantity          149116 non-null  int64  
 10  total_bill        149116 non-null  float64
dtypes: float64(2), int64(2), object(7)
memory usage: 12.5+ MB


In [4]:
# Drop Columns
drop_cols = ['transaction_id', # 100% cardinality
             'product_size']   # 29.85% not defined
df = df.drop(columns=drop_cols)

In [5]:
# Data Format
df['transaction_date'] = pd.to_datetime(df['transaction_date'], format='%d/%m/%Y')

In [6]:
# Rename Columns
df.columns = ['trn_date', 'trn_time', 'store', 'category', 'subcategory',
              'product', 'unit_price', 'quantity', 'total_bill']

In [7]:
# Sort Data
df.sort_values(by=['trn_date', 'trn_time'], inplace=True)
df = df.reset_index(drop=True)

In [8]:
# Outlier Detection
Q1 = df['total_bill'].quantile(0.25)
Q3 = df['total_bill'].quantile(0.75)
IQR = Q3 - Q1

lower_limit = Q1 - (IQR * 1.5)
upper_limit = Q3 + (IQR * 1.5)

outliers = df[(df['total_bill'] < lower_limit) | (df['total_bill'] > upper_limit)]
print(len(outliers))
# df = df.drop(outliers.index)

3273


In [9]:
# Create Features
# df['week_year'] = df['trn_date'].dt.strftime("%U")
# df['week_year'] = pd.to_numeric(df['trn_date'])

In [10]:
# Order Columns
# df = df[['trn_date', 'trn_time', 'week_year', 'store', 'category',
#          'subcategory', 'product', 'unit_price', 'quantity', 'total_bill']]

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149116 entries, 0 to 149115
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   trn_date     149116 non-null  datetime64[ns]
 1   trn_time     149116 non-null  object        
 2   store        149116 non-null  object        
 3   category     149116 non-null  object        
 4   subcategory  149116 non-null  object        
 5   product      149116 non-null  object        
 6   unit_price   149116 non-null  float64       
 7   quantity     149116 non-null  int64         
 8   total_bill   149116 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(5)
memory usage: 10.2+ MB


# Data Pipeline

In [10]:
data_raw = '../../data/raw/coffee_shop_sales.csv'
data_clean = '../../data/clean/coffee_shop_clean.csv'
data_train = '../../data/clean/coffee_shop_train.csv'
data_test = '../../data/clean/coffee_shop_test.csv'
data_line = '../../models/coffee_transform.joblib'

rnd_seed = 45
deadline = '2023-05-31'

drop_cols = ['transaction_id', 'product_size']
date_cols = ['transaction_date']
sort_data = ['transaction_date', 'transaction_time']
rename_cols = {'transaction_date':'trn_date',
               'transaction_time':'trn_time',
               'store_location':'store',
               'product_category':'category',
               'product_type':'subcategory',
               'product_detail':'product'}

## Create Pipeline

In [11]:
class DropColumns(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X.drop(columns=self.columns)

In [12]:
class ObjectToDate(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        for col in self.columns:
            X[col] = pd.to_datetime(X[col], format='%d/%m/%Y')
        return X

In [13]:
class SortData(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        X.sort_values(by=self.columns, inplace=True)
        return X.reset_index(drop=True)

In [14]:
class RenameColumns(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        X.rename(columns=self.columns, inplace=True)
        return X

In [15]:
data_transform = Pipeline([
    ('drop_cols', DropColumns(drop_cols)),
    ('date_format', ObjectToDate(date_cols)),
    ('sort_data', SortData(sort_data)),
    ('rename_cols', RenameColumns(rename_cols))
])

In [16]:
set_config(display='diagram')
display(data_transform)

Pipeline(steps=[('drop_cols',
                 DropColumns(columns=['transaction_id', 'product_size'])),
                ('date_format', ObjectToDate(columns=['transaction_date'])),
                ('sort_data',
                 SortData(columns=['transaction_date', 'transaction_time'])),
                ('rename_cols',
                 RenameColumns(columns={'product_category': 'category',
                                        'product_detail': 'product',
                                        'product_type': 'subcategory',
                                        'store_location': 'store',
                                        'transaction_date': 'trn_date',
                                        'transaction_time': 'trn_time'}))])

In [17]:
df_raw = pd.read_csv(data_raw)

data_transform_fit = data_transform.fit(df_raw)
df_raw_clean = data_transform_fit.transform(df_raw)

In [18]:
print(df_raw_clean.shape)
df_raw_clean.info()

(149116, 9)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149116 entries, 0 to 149115
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   trn_date     149116 non-null  datetime64[ns]
 1   trn_time     149116 non-null  object        
 2   store        149116 non-null  object        
 3   category     149116 non-null  object        
 4   subcategory  149116 non-null  object        
 5   product      149116 non-null  object        
 6   unit_price   149116 non-null  float64       
 7   quantity     149116 non-null  int64         
 8   total_bill   149116 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(5)
memory usage: 10.2+ MB


## Save and Test

In [19]:
joblib.dump(data_transform_fit, data_line)

data_pipeline = joblib.load(data_line)

try:
    data_pipeline
    print('Data pipeline is loaded...')

except Exception as e:
    print("Error:", str(e))

Data pipeline is loaded...


In [20]:
df_clean = data_pipeline.transform(df_raw)

df_clean.to_csv(data_clean, index=False)
df_raw_clean.iloc[[0,1,-2,-1]]

,trn_date,trn_time,store,category,subcategory,product,unit_price,quantity,total_bill
0,2023-01-01,07:06:11,Lower Manhattan,Coffee,Gourmet brewed coffee,Ethiopia,3.00,2,6.00
1,2023-01-01,07:08:56,Lower Manhattan,Tea,Brewed Chai tea,Spicy Eye Opener Chai,3.10,2,6.20
149114,2023-06-30,20:57:19,Hell's Kitchen,Coffee,Barista Espresso,Cappuccino,3.75,1,3.75
149115,2023-06-30,20:57:19,Hell's Kitchen,Flavours,Regular syrup,Hazelnut syrup,0.80,2,1.60
